# Metric
使用keras.metrics.Accuracy物件的update_state + result()方法來計算準確度

In [1]:
import tensorflow as tf
import numpy as np
m = tf.keras.metrics.Accuracy() 
_ = m.update_state([1, 2, 3, 4], [0, 2, 3, 4]) 

print(m.result().numpy()) 

# m.reset_states() 不重製的話  會接續下去算，也就是[1, 2, 3, 4, 1, 2, 3, 4] vs [0, 2, 3, 4, 0, 0, 3, 4]
_ = m.update_state([1, 2, 3, 4], [0, 0, 3, 4]) 

# sample weight = 0 代表不計入此sample
_ = m.update_state([1, 2, 3, 4], [0, 2, 3, 4], 
                    sample_weight=[1, 1, 0, 0]) 
print(m.result().numpy()) 


0.75
0.6


In [2]:
type(m)

tensorflow.python.keras.metrics.Accuracy

In [3]:
m = tf.keras.metrics.CategoricalAccuracy() 
# 前2個array但代表真實數據為第2類,第1類；後2筆數據代表預測機率最高的是第幾類，分別是第1類與第2類
# _ = m.update_state([[0, 0, 1], [0, 1, 0]], [[0.1, 0.9, 0.8], 
#                     [0.05, 0.95, 0]]) 
_ = m.update_state([[0, 0, 1], [0, 1, 0]], [[0.1, 0.1, 0.8], 
                    [0.5, 0.5, 0]]) 
print(m.result().numpy()) 

#m.reset_states() 

_ = m.update_state([[0, 1,0], [0, 1, 0]], [[0.1, 0.9, 0.8], 
                    [0.05, 0.95, 0]]) 
print(m.result().numpy()) 

0.5
0.75


In [4]:
m = tf.keras.metrics.SparseCategoricalAccuracy() 
# _ = m.update_state([[2], [1]], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]]) 
# _ = m.update_state([2,1], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]]) 
_ = m.update_state([1,1], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]]) 

print(m.result().numpy())

#m.reset_states() 

_ = m.update_state([[0], [0]], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]]) 
print(m.result().numpy())

1.0
0.5


In [5]:
m = tf.keras.metrics.CategoricalCrossentropy()
m.update_state([[0, 1, 0], [0, 0, 1]],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
'''
# EPSILON = 1e-7, y = y_true, y` = y_pred
# y` = clip_ops.clip_by_value(output, EPSILON, 1. - EPSILON)
# y` = [[0.05, 0.95, EPSILON], [0.1, 0.8, 0.1]]

# xent = -sum(y * log(y'), axis = -1)
#      = ((-log 0.95), (-log 0.1))
#      = [0.051, 2.302]
# Reduced mean = (0.051 + 2.302) / 2
'''
print(m.result().numpy()) 


#m.reset_states() 

m.update_state([[0, 1, 0], [0, 1, 0]],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])

print(m.result().numpy()) 

1.1769392
0.65707886


In [6]:
# 計算 cross entropy
# 注意加了Sparse代表原始數據沒有做one hot encoding
m = tf.keras.metrics.SparseCategoricalCrossentropy()
m.update_state(
  [1, 2],
  [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])

print(m.result().numpy())  

#m.reset_states() 

m.update_state(
  [0, 0],
  [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])

print(m.result().numpy())  

1.1769392
1.913049


# DNN神經網路訓練及驗證(TensorFlow版本) 

In [7]:
import numpy as np
import tensorflow as tf
tf.compat.v1.reset_default_graph()
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()

        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

num_epochs = 5
batch_size = 50
learning_rate = 0.001

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    

sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

batch 0: loss 2.393366
batch 1: loss 2.341737
batch 2: loss 2.235297
batch 3: loss 2.205807
batch 4: loss 2.175499
batch 5: loss 2.073968
batch 6: loss 2.057828
batch 7: loss 2.041076
batch 8: loss 1.808180
batch 9: loss 1.827261
batch 10: loss 1.794388
batch 11: loss 1.785298
batch 12: loss 1.768250
batch 13: loss 1.796226
batch 14: loss 1.600526
batch 15: loss 1.611787
batch 16: loss 1.595475
batch 17: loss 1.442492
batch 18: loss 1.325120
batch 19: loss 1.214459
batch 20: loss 1.413765
batch 21: loss 1.435347
batch 22: loss 1.168297
batch 23: loss 1.170470
batch 24: loss 1.142924
batch 25: loss 1.218995
batch 26: loss 1.050157
batch 27: loss 1.287215
batch 28: loss 1.001025
batch 29: loss 1.022929
batch 30: loss 0.952073
batch 31: loss 0.944396
batch 32: loss 0.858617
batch 33: loss 0.960282
batch 34: loss 0.761028
batch 35: loss 0.856271
batch 36: loss 0.793500
batch 37: loss 0.828553
batch 38: loss 0.817121
batch 39: loss 0.838593
batch 40: loss 0.803229
batch 41: loss 0.803657
ba